In this project, we are looking at Coronary artery disease, which is a common heart disease where the major blood vessels are clogged or narrowed. Due to the blockage, it is difficult to transfer oxygen, blood and nutrients to the heart. Usually, this disease can be attributed to cholesterol deposits, so the level of cholesterol in potential victims is of interest to us. In this project, we would like to determine whether a fictional person would be at risk of heart 

The dataset we chose was the one about heart disease. The database for heart disease contains 76 attributes (variables), but the dataset we are using includes only 14 of them. In our project, we will focus on determining the presence or absence of heart disease, so the data in the disease column has only two categories, 0 (no heart disease) and 1 (with heart disease). Also, this database has four regional datasets, and we finally chose the Hungarian regional. 

In [13]:
library(tidyverse)
library(repr)
options(repr.matrix.max.rows = 6)

In [14]:
#First step is to read the data and label the column names. 
heart_data <- read_delim("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data",col_name = FALSE)
colnames(heart_data) <- c("age", "sex", "chest_pain", "resting_bp", "cholesterol", "fbs>120mg/dL", "restingecg", "max_heart_rate", "exercise_induced_angina", "oldpeak", "slope", "ca", "thal", "disease")

Rows: 294 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): X4, X5, X6, X7, X8, X9, X11, X12, X13
dbl (5): X1, X2, X3, X10, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [15]:
#Here we are tidying the data by filtering out columns with "?" and selecting which columns we want. 
heart_data_tidy <- heart_data |> select(chest_pain, resting_bp, cholesterol, restingecg, max_heart_rate, disease) |>
                   filter(chest_pain!= "?", resting_bp!= "?", cholesterol!= "?", restingecg!= "?", max_heart_rate!= "?", disease!= "?") |>
                   mutate(chest_pain = as_factor(chest_pain), 
                          resting_bp = as.numeric(resting_bp),
                          cholesterol = as.numeric(cholesterol),
                          restingecg = as_factor(restingecg),
                          max_heart_rate = as.numeric(max_heart_rate),
                          disease = as_factor(disease))

In [16]:
heart_split <- initial_split(heart_data_tidy, prop = .75, strata = disease)  
heart_train <- training(heart_split)   
heart_test <- testing(heart_split)
heart_train

chest_pain,resting_bp,cholesterol,restingecg,max_heart_rate,disease
<fct>,<dbl>,<dbl>,<fct>,<dbl>,<fct>
2,130,132,2,185,0
2,120,243,0,160,0
1,170,237,1,170,0
⋮,⋮,⋮,⋮,⋮,⋮
3,130,294,1,100,1
4,155,342,0,150,1
4,130,275,1,115,1
